In [3]:
from datetime import datetime
import numpy as np
import numpy as numpy
import pandas as pd
import pylab
import calendar
from scipy import stats
import seaborn as sns
from sklearn import model_selection, preprocessing
from scipy.stats import kendalltau
import warnings
import matplotlib.pyplot as plt
import pandas

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from keras import callbacks
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg


C:\Users\Reshma.nspr\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Reshma.nspr\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Read Data

In [4]:
# Load train, Prop and sample
print('Loading train, prop and sample data')
train = pd.read_csv("C:/Users/Reshma.nspr/Desktop/ADS project/train_2016_v2.csv", parse_dates=["transactiondate"])
prop = pd.read_csv('C:/Users/Reshma.nspr/Desktop/ADS project/properties_2016.csv')
sample = pd.read_csv('C:/Users/Reshma.nspr/Desktop/ADS project/sample_submission.csv')

Loading train, prop and sample data


C:\Users\Reshma.nspr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print('Fitting Label Encoder on properties')
for c in prop.columns:
    prop[c]=prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))

Fitting Label Encoder on properties


In [6]:
#Create df_train and x_train y_train from that
print('Creating training set')
df_train = train.merge(prop, how='left', on='parcelid')

Creating training set


In [7]:
# Create df_test and test set
print('Creating df_test')
sample['parcelid'] = sample['ParcelId']

Creating df_test


In [8]:
print("Merge Sample with property data")
df_test = sample.merge(prop, on='parcelid', how='left')

Merge Sample with property data


Create TRAIN and TEST sets

In [9]:
INCLUDE_TIME_FEATURES = False
INCLUDE_SEASONAL_FEATURES = True
BASELINE_FUDGE_FACTOR = 1.033  # Optimal in XGB CV
FUDGE_FACTOR_SCALEDOWN = .25
N_EPOCHS = 150
BEST_EPOCH = False
CV_ONLY = False

In [10]:
df_train["transactiondate"] = pd.to_datetime(df_train["transactiondate"])
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter
if INCLUDE_TIME_FEATURES:
    df_train["transactiondate_year"] = df_train["transactiondate"].dt.year
    df_train["transactiondate_month"] = df_train["transactiondate"].dt.month
    df_train["transactiondate"] = df_train["transactiondate"].dt.day
if INCLUDE_SEASONAL_FEATURES:
    basedate = pd.to_datetime('2015-11-15').toordinal()
    df_train['cos_season'] = \
        ( (pd.to_datetime(df_train['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
          (2*np.pi/365.25) ).apply(np.cos)
    df_train['sin_season'] = \
        ( (pd.to_datetime(df_train['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
          (2*np.pi/365.25) ).apply(np.sin)

    
select_qtr4 = df_train["transactiondate_quarter"] == 4

#df_train =df_train[ df_train.logerror > -0.4005 ]
#df_train=df_train[ df_train.logerror < 0.412 ]

print('Create x_train and y_train from df_train' )
x_train_all = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode',
                             'fireplacecnt', 'fireplaceflag'], axis=1)
if not INCLUDE_TIME_FEATURES:
     x_train_all = x_train_all.drop(['transactiondate_quarter'], axis=1)

y_train_all = df_train["logerror"]
y_train = y_train_all[~select_qtr4]
x_train = x_train_all[~select_qtr4]
x_valid = x_train_all[select_qtr4]
y_valid = y_train_all[select_qtr4]

y_mean = np.mean(y_train)
print(x_train.shape, y_train.shape)
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

if INCLUDE_TIME_FEATURES:
    df_test["transactiondate"] = pd.to_datetime('2016-11-15')  # typical date for 2016 test data
    df_test["transactiondate_year"] = df_test["transactiondate"].dt.year
    df_test["transactiondate_month"] = df_test["transactiondate"].dt.month
    df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter
    df_test["transactiondate"] = df_test["transactiondate"].dt.day 
if INCLUDE_SEASONAL_FEATURES:
    basedate = pd.to_datetime('2015-11-15').toordinal()
    df_test['cos_season'] = np.cos( (pd.to_datetime('2016-11-15').toordinal() - basedate) * \
                                    (2*np.pi/365.25) )
    df_test['sin_season'] = np.sin( (pd.to_datetime('2016-11-15').toordinal() - basedate) * \
                                    (2*np.pi/365.25) )

Create x_train and y_train from df_train
(81733, 55) (81733,)


In [11]:
x_test = df_test[train_columns]

print('Shape of x_test:', x_test.shape)
print("Preparing x_test:")
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

Shape of x_test: (2985217, 55)
Preparing x_test:


In [12]:
from sklearn.preprocessing import Imputer
imputer= Imputer()
imputer.fit(x_train.iloc[:, :])
x_train = imputer.transform(x_train.iloc[:, :])
imputer.fit(x_valid.iloc[:, :])
x_valid = imputer.transform(x_valid.iloc[:, :])
imputer.fit(x_test.iloc[:, :])
x_test = imputer.transform(x_test.iloc[:, :])

Standard Scalar

In [13]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_valid = sc.fit_transform(x_valid)
x_val = np.array(x_valid)
y_val = np.array(y_valid)

Train Network

In [14]:
len_x=int(x_train.shape[1])
print("len_x is:",len_x)

nn = Sequential()
nn.add(Dense(units = 400 , kernel_initializer = 'normal', input_dim = len_x))
nn.add(PReLU())
nn.add(Dropout(.4))
nn.add(Dense(units = 160 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(units = 64 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))
nn.add(Dense(units = 26, kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(1, kernel_initializer='normal'))

nn.compile(loss='mae', optimizer=Adam(lr=5e-3,decay=1e-4))

wtpath = 'weights.hdf5'
bestepoch = ModelCheckpoint( filepath=wtpath, verbose=1, save_best_only=True )

nn.fit(np.array(x_train), np.array(y_train), batch_size = 32, epochs = N_EPOCHS, verbose=2, 
       validation_data=(x_val,y_val), callbacks=[bestepoch])

if BEST_EPOCH:       
    nn.load_weights( wtpath )

valid_pred = nn.predict(x_val)

len_x is: 55
Train on 81733 samples, validate on 8542 samples
Epoch 1/150
 - 19s - loss: 0.0707 - val_loss: 0.0656

Epoch 00001: val_loss improved from inf to 0.06561, saving model to weights.hdf5
Epoch 2/150
 - 15s - loss: 0.0685 - val_loss: 0.0660

Epoch 00002: val_loss did not improve
Epoch 3/150
 - 15s - loss: 0.0684 - val_loss: 0.0654

Epoch 00003: val_loss improved from 0.06561 to 0.06545, saving model to weights.hdf5
Epoch 4/150
 - 15s - loss: 0.0683 - val_loss: 0.0660

Epoch 00004: val_loss did not improve
Epoch 5/150
 - 15s - loss: 0.0683 - val_loss: 0.0661

Epoch 00005: val_loss did not improve
Epoch 6/150
 - 15s - loss: 0.0682 - val_loss: 0.0658

Epoch 00006: val_loss did not improve
Epoch 7/150
 - 15s - loss: 0.0682 - val_loss: 0.0655

Epoch 00007: val_loss did not improve
Epoch 8/150
 - 15s - loss: 0.0681 - val_loss: 0.0660

Epoch 00008: val_loss did not improve
Epoch 9/150
 - 16s - loss: 0.0681 - val_loss: 0.0653

Epoch 00009: val_loss improved from 0.06545 to 0.06526, sa


Epoch 00084: val_loss did not improve
Epoch 85/150
 - 14s - loss: 0.0674 - val_loss: 0.0652

Epoch 00085: val_loss did not improve
Epoch 86/150
 - 15s - loss: 0.0674 - val_loss: 0.0653

Epoch 00086: val_loss did not improve
Epoch 87/150
 - 14s - loss: 0.0674 - val_loss: 0.0655

Epoch 00087: val_loss did not improve
Epoch 88/150
 - 14s - loss: 0.0674 - val_loss: 0.0653

Epoch 00088: val_loss did not improve
Epoch 89/150
 - 14s - loss: 0.0674 - val_loss: 0.0654

Epoch 00089: val_loss did not improve
Epoch 90/150
 - 14s - loss: 0.0674 - val_loss: 0.0653

Epoch 00090: val_loss did not improve
Epoch 91/150
 - 14s - loss: 0.0673 - val_loss: 0.0654

Epoch 00091: val_loss did not improve
Epoch 92/150
 - 14s - loss: 0.0674 - val_loss: 0.0655

Epoch 00092: val_loss did not improve
Epoch 93/150
 - 14s - loss: 0.0674 - val_loss: 0.0656

Epoch 00093: val_loss did not improve
Epoch 94/150
 - 14s - loss: 0.0674 - val_loss: 0.0654

Epoch 00094: val_loss did not improve
Epoch 95/150
 - 14s - loss: 0.0

Optimize Fudge Factor

In [15]:
print("\nMean absolute validation error with baseline fudge factor: ")
print(mean_absolute_error(y_valid, BASELINE_FUDGE_FACTOR*valid_pred))

mod = QuantReg(y_valid, valid_pred)
res = mod.fit(q=.5)
print("\nLAD Fit for Fudge Factor:")
print(res.summary())
fudge = res.params[0]
print("Optimized fudge factor:", fudge)

print("\nMean absolute validation error with optimized fudge factor: ")
print(mean_absolute_error(y_valid, fudge*valid_pred))

fudge **= FUDGE_FACTOR_SCALEDOWN

print("Scaled down fudge factor:", fudge)

print("\nMean absolute validation error with scaled down fudge factor: ")
print(mean_absolute_error(y_valid, fudge*valid_pred))

print("\nx_test.shape:",x_test.shape)


Mean absolute validation error with baseline fudge factor: 
0.06527530656370696

LAD Fit for Fudge Factor:
                         QuantReg Regression Results                          
Dep. Variable:               logerror   Pseudo R-squared:             0.007729
Model:                       QuantReg   Bandwidth:                     0.01105
Method:                 Least Squares   Sparsity:                       0.1037
Date:                Sat, 21 Apr 2018   No. Observations:                 8542
Time:                        05:58:10   Df Residuals:                     8541
                                        Df Model:                            1
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.3676      0.049     27.704      0.000       1.271       1.464
Optimized fudge factor: 1.3675743030930692

Mean absolute validation error with optimized fudge factor

Predict and Write Results

In [16]:
if not CV_ONLY:
    print("\nGenerating predictions...")
    y_pred_ann = fudge*nn.predict(x_test)

    print( "\nPreparing results for write..." )
    y_pred = y_pred_ann.flatten()

    output = pd.DataFrame({'ParcelId': prop['parcelid'].astype(np.int32),
            '201610': y_pred, '201611': y_pred, '201612': y_pred,
            '201710': y_pred, '201711': y_pred, '201712': y_pred})
    # set col 'ParceID' to first col
    cols = output.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    output = output[cols]

    print( "\nWriting results to disk:" )
    output.to_csv('Only_ANN_{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)



print( "\nFinished!" )


Generating predictions...

Preparing results for write...

Writing results to disk:

Finished!
